## Gabriela Osorio - Data Science Immersive Project 4

#### GOAL: Predict locations where the City of Chicago should be spraying anti-mosquito spray in 2014

source: https://www.kaggle.com/c/predict-west-nile-virus/data

----------
**Workflow Organization**

 **Phase 1 Describe** - Using datasets, identify any patterns within and between features
    - Preprocessing - making data functional
    - Exploratory Data Analysis
    
 **Phase 2 Predict** - Using data patterns, and algorithms, predict WNV locations
    - Modeling 
        * Spatial-temporal Clustering?
        * 
 **Phase 3 Prescribe** - Using predictions, suggest future spray locations and times.
    - Algorithm execution


-------------------
**Phase 1 Describe**

Preprocessing:
* Loading the datasets: 'train', 'weather', 'spray'
* Exploring the datasets' contents: columns? datatypes? null values?
* Transforming the datasets: objects to floats or pandas datetime

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer

In [ ]:
train=pd.read_csv('./data/train.csv')
train.head(1)

In [ ]:
train.dtypes

In [ ]:
weather=pd.read_csv('./data/weather.csv')
print(weather.columns)
print(weather.dtypes)
weather.head(1)

In [ ]:
weather.Station.values

In [ ]:
weather.isnull().sum()

In [ ]:
spray=pd.read_csv('./data/spray.csv')
print(spray.columns)
print(spray.dtypes)
spray.head(1)

In [ ]:
spray['Date'].head()

In [ ]:
from datetime import datetime

In [ ]:
spray['Date'].head()

In [ ]:
train['Date']=pd.to_datetime(train['Date'])
weather['Date']=pd.to_datetime(weather['Date'])
spray['Date']=pd.to_datetime(spray['Date'])
spray['Time']=pd.to_datetime(spray['Time'])


In [ ]:
lb=LabelBinarizer()
WnvPresent=lb.fit(train['WnvPresent'])
WnvPresent

In [ ]:
speciesdf = pd.crosstab(train.Date, train.Species).astype(bool).astype(int)
speciesdf

-------------------
**Phase 1 Describe**(cont.)

Exploratory Date Analysis:
* Culling the dataset, removing features that lack scientific literature outlining a relationship with mosquitos
* Exploring relationships within data using bar graphs, scatter plots
* Feature Engineering

We will keep these:

**Train**
- Date
- Address             
- Trap
- NumMosquitos
- Species
- WnVPresent

**Weather**
- Date
- Tavg
- Wetbulb
- Heat
- Sunset-Sunrise
- Water


In [ ]:
weather=weather.drop(columns=['Station', 'Tmax', 'Tmin','Depart', 
        'CodeSum', 'Depth',
       'SnowFall','ResultSpeed', 'AvgSpeed'])

In [ ]:
train=train.drop(columns=['Address','Street', 'Block',
       'AddressNumberAndStreet',  'AddressAccuracy'])


Now let's begin the EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_palette('magma_r')
sns.set_style("whitegrid")

In [ ]:
ax = sns.boxplot(y="NumMosquitos",x="WnvPresent", data=train,showfliers=False,palette='magma_r')
plt.show()

In [ ]:
mosquitoprob = train.groupby(pd.cut(train['NumMosquitos'], range(0, 50,5)), as_index = False).WnvPresent.mean()
mosquitoprob['NumMosquitos']=[5,10,15,20,25,30,35,40,45]

In [ ]:
sns.barplot( x="NumMosquitos",y="WnvPresent", data=mosquitoprob, palette='magma_r' )

In [ ]:
# species=speciesdf.columns
# result = pd.merge(speciesdf, train)
# result.head(2)
speciesdf.head(2)

In [ ]:
ax = sns.barplot( y="Species",x="WnvPresent", data=train, palette='magma_r')
ax.set_yticklabels

plt.show()

In [ ]:
ax = sns.barplot(y="Species", x="NumMosquitos", hue="WnvPresent", data=train, palette="magma_r")

In [ ]:
mosquito_type = train.groupby(train['Species'], as_index = False).WnvPresent.sum()
ax = sns.barplot( x="Species",y="WnvPresent", data=mosquito_type, palette='magma_r')
ax.set_xticklabels(mosquito_type.Species,rotation=90)
plt.show()

In [ ]:
train['Month'] = pd.to_datetime(train['Date'])
train['Month'] = train['Date'].dt.month
train['Month']=train['Month'].replace({5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October'})
sns.barplot( x="Month", y="WnvPresent", data=train, palette="magma_r");

In [ ]:
train['Year'] = pd.to_datetime(train['Date'])
train['Year'] = train['Date'].dt.year
#train['Datey']=train['Date'].replace({5:'May', 6:'June', 7:'July', 8:'August', 9:'September', 10:'October'})
sns.barplot( x="Year", y="WnvPresent", data=train, palette="magma_r");

Graphing Maps

In [ ]:
mapdata_copyright_openstreetmap_contributors.txt

In [ ]:
mapdata_copyright_openstreetmap_contributors.rds

In [ ]:
'data/geo_export_b249b76e-3935-445d-90ad-85874824b258.shp'

In [2]:
from pysal.contrib.viz import mapping as viz


/Applications/anaconda3/lib/python3.6/site-packages/pysal/__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [4]:
shp_link = './data/geo_export_b249b76e-3935-445d-90ad-85874824b258.shp'
viz.plot_poly_lines('./data/geo_export_b249b76e-3935-445d-90ad-85874824b258.shp')
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: './data/geo_export_b249b76e-3935-445d-90ad-85874824b258.shx'

<Figure size 432x288 with 0 Axes>

In [ ]:
shp_link = './data/geo_export_b249b76e-3935-445d-90ad-85874824b258.shp'
values = np.array(ps.open('../data/texas.dbf').by_col('HR90'))

types = ['classless', 'unique_values', 'quantiles', 'equal_interval', 'fisher_jenks']
for typ in types:
    maps.plot_choropleth(shp_link, values, typ, title=typ)

In [ ]:
from pysal.contrib.viz import mapping as viz

viz.plot_choropleth(,
                    values = pci2009,
                    type = 'quantiles',
                    cmap='viridis')

# NOTES:

**Content-Related Info to Consider:**
-  how long is transmission? how long does the virus take spread? how long is the virus dormant/active?
- how else is the illness transmitted
- how much does the anti-mosquito cost?
- who gets the virus most? -- extra dataset
    
  

- Choosing to call the mosquito killer 'anti mosquito' instead.

- Choosing to apply a Rolling Means Function, as timeseries closer to predictive event is where the most pertinent and correlated info is encountered

- Gonna do that thing that comes up with an area graph that shows which dates are relevant? 

Based on the research we focused our feature engineering on patterns of moisture and heat over time. For moisture, we looked at total precipitation and average wet bulb over time, theorizing that precipitation represents an increase in moisture, while wet bulb is an indicator of evaporation rate, thus decreases in moisture. For heat, we worked with cumulative cooling degree days over time. For each of these metrics, we created new variables for the last 7 days, the last 30 days, and the last 90 days. For the 30 and 90 day variables, we subtracted the 7 and 30 day variables respectively for greater feature independence.

- We created visualizations of these vs. WNV presence over time (example below.) Although some of the dips in the visualizations are due to the timing of the data samples, there appears to be correlation between the variables created and WNV presence.

- While the precipitation, wet bulb, and cooling degree day features were being created, we trained and tested models using multiple algorithms to get a baseline of model performance. Then, when we added these features, we were able to see improvements to performance.

- We then examined feature importances to do variable selection. We achieved our best models using some of the engineered variables (30 and 90 day precipitation and cooling degree days), while removing others (wet bulb and 7 day precipitation and cooling) utilizing an XGBoost model.

Our final AUC-ROC scores were above 0.78, up from about 0.69 in our baseline models.



- EDA involves exploring and plotting the yearly trends of overall transmission, monthly trends, 
